In [12]:
import pandas as pd
import spacy
import json
from tqdm import tqdm

In [13]:
df = pd.read_json('../data/out.json')
df.head()

,title,author,rating,ISBN,genres,pageCount,url,quotes
0,Pretty Little Liars,Sara Shepard,3.86,9.780061e+12,"[Young Adult, Mystery, Contemporary, Fiction, ...",286.0,https://www.goodreads.com/book/show/162085.Pre...,"[{'text': '“I'm still here, bitches. And I kno..."
1,Ringworld,Larry Niven,3.95,9.780575e+12,"[Science Fiction, Fiction, Science Fiction Fan...",288.0,https://www.goodreads.com/book/show/61179.Ring...,[{'text': '“The gods do not protect fools. Foo...
2,Lover Enshrined,J.R. Ward,4.11,9.780451e+12,"[Romance, Paranormal, Paranormal Romance, Vamp...",560.0,https://www.goodreads.com/book/show/1240662.Lo...,"[{'text': '“After a moment, Wrath turned to Jo..."
3,Blood Red Road,Moira Young,3.91,9.781442e+12,"[Young Adult, Dystopia, Fantasy, Romance, Scie...",459.0,https://www.goodreads.com/book/show/9917938-bl...,"[{'text': '“Yer in my blood, Saba, he says. Ye..."
4,"Eragon, Eldest & Brisingr",Christopher Paolini,4.24,9.780376e+12,"[Fantasy, Young Adult, Fiction, Dragons, Magic...",1952.0,https://www.goodreads.com/book/show/2479827.Er...,"[{'text': '“Ah, pay no heed if your enemies la..."


In [14]:
quote_df = pd.DataFrame(pd.Series(quote for quote_list in df['quotes'] for quote in quote_list)).rename(columns={0:'json'})

quote_df['text'] = quote_df['json'].map(lambda x : x['text'])
quote_df['likes'] = quote_df['json'].map(lambda x : x['likes'])
quote_df['tags'] = quote_df['json'].map(lambda x : x['tags'])
quote_df = quote_df.drop('json', axis=1)
quote_df.head()

,text,likes,tags
0,"“I'm still here, bitches. And I know everythin...",1053,[defiance]
1,“You know what they say about hope. It breeds ...,294,[spencer]
2,"“I kind of have to go to the bathroom,"" Aria s...",201,"[humour, lust]"
3,“Only two can keep a secret if one of them is ...,196,[]
4,“When someone covets something they desire and...,148,[]


In [16]:
quote_df.describe()

,likes
count,855492.000000
mean,33.338041
std,391.477813
min,-9.000000
25%,0.000000
50%,1.000000
75%,8.000000
max,99594.000000


In [10]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def append_row(df, row):
    return pd.concat([
                df, 
                pd.DataFrame([row], columns=row.index)]
           ).reset_index(drop=True)

df = pd.DataFrame(columns=('text', 'startChar', 'endChar', 'label'))

for doc in tqdm(nlp.pipe(quote_df['text'].values), total=855492):
    for ent in doc.ents:
        new_row = pd.Series({'text':ent.text, 'startChar':ent.start_char, 'endChar': ent.end_char, 'label': ent.label_})
        append_row(df, new_row)

  3%|█                                  | 24509/855492 [01:57<57:07, 242.44it/s]